In [204]:
### 라이브러리 임포트

import pandas as pd
import re
import time
import requests
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup

In [205]:
### 헤더, 함수 정의

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Connection": "keep-alive",
        "Cache-Control": "max-age=0",
        "sec-ch-ua-mobile": "?0",
        "DNT": "1",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ko-KR,ko;q=0.9"
    }

def get_soup(url, time_sleep=0, max_retries=600):
    try:
        if max_retries <= 0:
            print("[최대 재시도 횟수 초과 : get_soup()]")
            return None
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            time.sleep(time_sleep)
        soup = BeautifulSoup(response.text, "html.parser")
        if len(soup) == 0:  # 불러오는데 실패하면
            print(f"[soup 로딩 실패, 반복] [get_soup(time_sleep=1)]")
            soup = get_soup(url, 1)
    except Exception as e:
        print(f"[오류 발생, 반복] [get_soup(time_sleep=1)] ", e)
        soup = get_soup(url, 1, max_retries-1)
    return soup

In [ ]:
"""
[프로세스]

1. 하나의 tourney 대상으로 메타 분석
    play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame

    step a) 아키타입 별 상대 승률
    step b) 20장 덱 유형 별 vs 아키타입 승률
    step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)

2. 전체 tourney 묶어서 다시 분석

    step a) Standard한 Tourney 분류
            포맷을 Standard로 설정해도, 스탠다드 룰이 아닌 Tourney가 있음. 
            
"""

In [17]:
### 파일 불러오기

In [206]:
tourney_result_file_name = "tourney_result_250118_040230.csv"
df = pd.read_csv("./tourney_result/" + tourney_result_file_name, encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          133 non-null    object
 1   tourney_name  133 non-null    object
 2   organizer     133 non-null    object
 3   players       133 non-null    int64 
 4   winner        133 non-null    object
 5   country       133 non-null    object
 6   url           133 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.4+ KB


In [207]:
df.head()

,date,tourney_name,organizer,players,winner,country,url
0,2025-01-17,IDEAVOLUTION Swiss Round Tournament,PTCGO-ID,333,kana41,Japan,play.limitlesstcg.com/tournament/677ff37e7960e...
1,2025-01-17,Flash Friday #4,Pokemon Downunder,163,Manbok,Korea,play.limitlesstcg.com/tournament/67848af07960e...
2,2025-01-17,Torneio TD PADRÃO - 2#/Janeiro - Free - BR,Thiago Donola,156,00_kj7,Japan,play.limitlesstcg.com/tournament/6780a9c67960e...
3,2025-01-17,Team Rocket Pocket Cup #25,Team Rocket,127,Manaa,Japan,play.limitlesstcg.com/tournament/6774a709ed2c7...
4,2025-01-17,Team Rocket Pocket Cup #24,Team Rocket,211,bau,Japan,play.limitlesstcg.com/tournament/6774a6d7ed2c7...


In [9]:
# 1. 실험 : 하나의 tourney 대상으로 메타 분석

In [ ]:
### 저장할 데이터베이스 초기화 : 2중 딕셔너리로 저장할 예정
tourney_data = {}

In [334]:
### 샘플 토너먼트 url 설정
# tourney_meta_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame"
tourney_meta_url = "https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame"
limitlesstcg_url_base = "https://play.limitlesstcg.com"

In [201]:
## step a) 아키타입 별 상대 승률

In [335]:
### 전처리 함수 : 덱 타입
def preprocess_deck_type(deck_type):
    processed_deck_type = deck_type.lower()  # 소문자화
    processed_deck_type = re.sub(r'[^a-zA-Z ]', '', processed_deck_type)  # 영어 알파벳(대소문자)과 공백만 남기고 모두 제거
    return processed_deck_type.replace(' ', '_')  # 공백문자를 _로 변경

### bs4로 html 가져오기
soup = get_soup(tourney_meta_url)

### 정보 추출 : 덱 타입
tourney_deck_type_list = []
min_deck_count = 1  # 최소 덱 타입 수

meta_tr_list = soup.select('table.meta tr')
for tr in meta_tr_list[1:]:
    # 정보 추출
    deck_count = int(tr.select('td')[1].text)
    if deck_count < min_deck_count:
        continue
    deck_type = preprocess_deck_type(tr.select('td')[2].text)
    deck_url = limitlesstcg_url_base + tr.select('td')[2].find('a')['href']
    print(deck_count, deck_type, deck_url)

    # DB에 덱타입 추가 : 기존 정보가 있는지 확인
    if deck_type not in tourney_data:
        tourney_data[deck_type] = {}
    
    # df에 저장
    row = [deck_type, deck_count, deck_url]
    tourney_deck_type_list.append(row)

# df : 덱 타입 - 중복 덱타입 존재함.
df_tourney_deck_type_list  = pd.DataFrame(tourney_deck_type_list, columns=["deck_type", "deck_count", "deck_url"])

328 mewtwo_ex_gardevoir https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/mewtwo-ex-gardevoir-a1
237 pikachu_ex_zebstrika https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/pikachu-ex-zebstrika-a1
175 gyarados_ex_greninja https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/gyarados-ex-a1a-greninja-a1
103 exeggutor_ex_celebi_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/exeggutor-ex-a1-celebi-ex-a1a
73 charizard_ex_moltres_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-moltres-ex-a1
63 celebi_ex_serperior https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/celebi-ex-serperior-a1a
52 charizard_ex_arcanine_ex https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/charizard-ex-arcanine-ex-a1
35 aerodactyl_ex_primeape https://play.limitlesstcg.com/tournament/678305977960ec710eff1898/metagame/aerodactyl-

In [336]:
len(tourney_data)

251

In [337]:
len(tourney_deck_type_list)  
df_tourney_deck_type_list

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,328,https://play.limitlesstcg.com/tournament/67830...
1,pikachu_ex_zebstrika,237,https://play.limitlesstcg.com/tournament/67830...
2,gyarados_ex_greninja,175,https://play.limitlesstcg.com/tournament/67830...
3,exeggutor_ex_celebi_ex,103,https://play.limitlesstcg.com/tournament/67830...
4,charizard_ex_moltres_ex,73,https://play.limitlesstcg.com/tournament/67830...
...,...,...,...
163,aerodactyl_ex_weezing,1,https://play.limitlesstcg.com/tournament/67830...
164,primeape_kabutops,1,https://play.limitlesstcg.com/tournament/67830...
165,moltres_ex_volcarona,1,https://play.limitlesstcg.com/tournament/67830...
166,zapdos_ex_raichu,1,https://play.limitlesstcg.com/tournament/67830...


In [318]:
### 덱 타입 필터링 
# min_deck_count = 300
# df_tourney_deck_type_filtered = df_tourney_deck_type_list[df_tourney_deck_type_list['deck_count'] >= min_deck_count]
# df_tourney_deck_type_filtered.tail()

,deck_type,deck_count,deck_url
0,mewtwo_ex_gardevoir,302,https://play.limitlesstcg.com/tournament/6779b...


In [338]:
### 덱 타입 별 

for index, row in df_tourney_deck_type_list.iterrows():
    print(f"index : {index}")
    print(row['deck_type'])
    matchup_deck_type = row['deck_type']
    matchup_url = row['deck_url'] + "/matchups"
    matchup_soup = get_soup(matchup_url)

    ### 정보 추출 : 덱 별 매치업 통계
    matchup_tr_list = matchup_soup.select('table.striped tr')[1:-1]
    
    for tr in matchup_tr_list:
        # 정보 추출
        opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
        win_count = int(tr.select('td')[2].text.split(' - ')[0])
        lose_count = int(tr.select('td')[2].text.split(' - ')[1])
        print(opponent_deck_type, win_count, lose_count)
    
        # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
        try:
            tourney_data[matchup_deck_type][opponent_deck_type]
        except: 
            tourney_data[matchup_deck_type][opponent_deck_type] = {"win_count": 0, "lose_count": 0}
    
        # 3. 승리/패배 횟수 누적
        tourney_data[matchup_deck_type][opponent_deck_type]["win_count"] += win_count
        tourney_data[matchup_deck_type][opponent_deck_type]["lose_count"] += lose_count

        # print(tourney_data[matchup_deck_type])

print("[done] ")

index : 0
mewtwo_ex_gardevoir
Blastoise_ex_Starmie_ex 2 0
Alakazam_Mew_ex 2 0
Zebstrika_Pikachu_ex 2 0
Wigglytuff_ex_Exeggutor_ex 2 0
Dragonite_Zebstrika 2 0
Machamp_ex_Aerodactyl_ex 2 0
Mewtwo_ex_Alakazam 2 0
Gyarados_ex_Kabutops 2 0
Greninja_Hitmonlee 1 0
Golbat_Mew_ex 1 0
Marowak_ex_Sandslash 1 0
Beheeyem_Mew_ex 1 0
Moltres_ex_Rapidash 1 0
Arcanine_ex_Salazzle 1 0
Machamp_ex_Druddigon 1 0
Alakazam_Jynx 1 0
Golem_Primeape 1 0
Gardevoir_Mewtwo 1 0
Gengar_ex_Druddigon 1 0
Alakazam_Clefable 1 0
Greninja_Melmetal 1 0
Golem_Farfetch'd 1 0
Aerodactyl_ex_Primeape 1 0
Mew_ex_Greninja 1 0
Machamp_ex_Mew_ex 1 0
Raichu_Zebstrika 1 0
Mew_ex_Golem 1 0
Alakazam_Florges 1 0
Machamp_ex_Primeape 1 0
Wigglytuff_ex_Weezing 1 0
Gyarados_ex_Chatot 1 0
Arcanine_ex_Greninja 1 0
Exeggutor_ex_Butterfree 1 0
Moltres_ex_Volcarona 1 0
Blastoise_ex_Druddigon 1 0
Machamp_ex_Hitmonlee 1 0
Golem_Druddigon 14 2
Pidgeot_ex_Hitmonlee 5 1
Ninetales_Rapidash 4 1
Charizard_ex_Moltres_ex 44 17
Aerodactyl_ex_Primeape 24 10

In [339]:
tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 7, 'lose_count': 2},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 4, 'lose_count': 2},
  'Pidgeot_ex_Hitmonlee': {'win_count': 7, 'lose_count': 1},
  'Aerodactyl_ex_Onix': {'win_count': 3, 'lose_count': 1},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 4, 'lose_count': 1},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 3, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 1},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

In [328]:
# ### 테스트용 (위에 덧댈거임)
# matchup_deck_type = 'Mewtwo_ex_Gardevoir'
# matchup_url = "https://play.limitlesstcg.com/tournament/6779bb0bed2c78098c6b086b/metagame/mewtwo-ex-gardevoir-a1" + "/matchups"
# matchup_soup = get_soup(matchup_url)

# ### 정보 추출 : 덱 별 매치업 통계
# tr_list = soup.select('table.striped tr')[1:-1]

# for tr in tr_list:
#     # 정보 추출
#     opponent_deck_type = tr.select('td')[1].text.replace(' ', '_')
#     win_count = int(tr.select('td')[2].text.split(' - ')[0])
#     lose_count = int(tr.select('td')[2].text.split(' - ')[1])
#     print(opponent_deck_type, win_count, lose_count)

#     # 정보 저장 : opponent_deck_type이 tourney_data[deck_type]에 있는지 확인 (없으면 추가)
#     if opponent_deck_type not in tourney_data[deck_type][opponent_deck_type]:
#         tourney_data[matchup_deck_type][opponent_deck_type] = {"승리": 0, "패배": 0}

#     # 3. 승리/패배 횟수 누적
#     tourney_data[matchup_deck_type][opponent_deck_type]["승리"] += win_count
#     tourney_data[matchup_deck_type][opponent_deck_type]["패배"] += lose_count

In [331]:
backup_tourney_data = tourney_data
backup_tourney_data

{'mewtwo_ex_gardevoir': {'Golem': {'win_count': 4, 'lose_count': 0},
  'Alakazam_Beheeyem': {'win_count': 4, 'lose_count': 0},
  'Exeggutor_ex_Greninja': {'win_count': 3, 'lose_count': 0},
  'Pidgeot_ex_Hitmonlee': {'win_count': 2, 'lose_count': 0},
  'Aerodactyl_ex_Onix': {'win_count': 2, 'lose_count': 0},
  'Articuno_ex_Mew_ex': {'win_count': 2, 'lose_count': 0},
  "Mew_ex_Farfetch'd": {'win_count': 2, 'lose_count': 0},
  'Greninja_Marshadow': {'win_count': 2, 'lose_count': 0},
  'Mew_ex_Alakazam': {'win_count': 2, 'lose_count': 0},
  'Moltres_ex_Rapidash': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Aerodactyl_ex': {'win_count': 1, 'lose_count': 0},
  'Marowak_ex_Primeape': {'win_count': 2, 'lose_count': 0},
  'Marowak_ex_Golem': {'win_count': 1, 'lose_count': 0},
  "Golem_Farfetch'd": {'win_count': 1, 'lose_count': 0},
  'Machamp_ex_Mew_ex': {'win_count': 1, 'lose_count': 0},
  'Pidgeot_ex_Greninja': {'win_count': 1, 'lose_count': 0},
  'Mew_ex_Mewtwo_ex': {'win_count': 1, 'lo

In [ ]:
## step b) 20장 덱 유형 별 vs 아키타입 승률

In [19]:
## step c) 아키타입 별 승률 높은 20장 덱 유형 분석 (궁극의 덱이 무엇일까)